In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as onp
import jax
import jax.numpy as jnp
import flax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import torch
import matplotlib
import timecast as tc
import inspect

from mpl_toolkits import mplot3d


plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

In [6]:
from typing import Tuple
from timecast.utils import random

class NewMixin:
    # TODO (flax)
    @classmethod
    def new(cls, shape: Tuple[int, ...], key: jnp.ndarray = None, name: str = None, *, **kwargs):
        argspec = inspect.getfullargspec(cls.apply)
        
        num_kwargs = len(argspec.defaults)
            
        _kwargs = {}
        
        # Capture kwarg defaults
        for i in range(1, num_kwargs + 1):
            _kwargs[argspec.args[-i]] = argspec.defaults[-i]
        
        # Override kwarg defaults, if any
        for kw, arg in kwargs.items():
            if kw in argspec.args or argspec.varkw is not None:
                _kwargs[kw] = arg
            else:
                raise ValueError("Found extra kwarg {}: {}".format(kw, arg))
        
        model_def = cls.partial(**kwargs)
        
        if name is None:
            name = cls.__name__.lower()
            
        if key is None:
            key = random.generate_key()
        
        with flax.nn.stateful() as state:
            _, params = model_def.init_by_shape(key, [shape])
        model = flax.nn.Model(model_def, params)
        
        return model, state
        
        

In [14]:
class Dummy(NewMixin, flax.nn.Module):
    def apply(self, x, a=1, b=2):
        print(type(x), type(a), type(b))
        print(x, a, b)
        return x + a + b

In [16]:
model_def = Dummy.partial()
_, params = model_def.init_by_shape(jax.random.PRNGKey(0), [(1, 2)])

TypeError: partial() takes 1 positional argument but 2 were given

In [11]:
Dummy.new(shape=(1,2), b=3)[0](4)

dummy [1194623263 2038155241] <class 'flax.nn.base.Dummy'> (1, 2)
<class 'jax.interpreters.partial_eval.JaxprTracer'> <class 'int'> <class 'int'>
Traced<ShapedArray(float32[1,2]):JaxprTrace(level=0/0)> 1 3
FullArgSpec(args=['self', 'x', 'a', 'b'], varargs=None, varkw=None, defaults=(2, 1, 2), kwonlyargs=[], kwonlydefaults=None, annotations={}) <class 'flax.nn.base.Dummy'>
<class 'int'> <class 'int'> <class 'int'>
4 1 3


8

In [ ]:
a = [1,2,3]

In [ ]:
a[-0]

In [ ]:
a[-1]

In [ ]:
a[-3]